## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Table of Contents
* [Step 1. Open the data file and have a look at the general information](#chapter1)
* [Step 2. Data preprocessing](#chapter2)
* [Step 3. Answer these questions](#chapter3)
* [Step 4. General conclusion](#chapter4)


### Step 1. Open the data file and have a look at the general information. 
<a class="anchor" id="chapter1"></a>

In [1]:
import pandas as pd
import random
from math import trunc

import warnings

warnings.filterwarnings ("ignore")

loanTable = pd.read_csv('/Users/yoni/Pictures/jupyter/Data_prossesing_1st/credit_scoring_eng.csv')
loanTable.head(15)
#loanTable.info()
#loanTable.describe()
loanTable.head(20)

loanTable.describe(include='object'),loanTable.describe()

(                  education family_status gender income_type           purpose
 count                 21525         21525  21525       21525             21525
 unique                   15             5      3           8                38
 top     secondary education       married      F    employee  wedding ceremony
 freq                  13750         12380  14236       11119               797,
            children  days_employed     dob_years  education_id  \
 count  21525.000000   19351.000000  21525.000000  21525.000000   
 mean       0.538908   63046.497661     43.293380      0.817236   
 std        1.381587  140827.311974     12.574584      0.548138   
 min       -1.000000  -18388.949901      0.000000      0.000000   
 25%        0.000000   -2747.423625     33.000000      1.000000   
 50%        0.000000   -1203.369529     42.000000      1.000000   
 75%        1.000000    -291.095954     53.000000      1.000000   
 max       20.000000  401755.400475     75.000000      4.000000

I see in the table a lot of information, some of it is confusing, like the "days_employed" column. my task is to understand if a persons merital status and number of children has an impact on whether they will default on a loan. and at first glance a lot of the info in this chart is not relavent to my task. i shall take a deeper look in this table and see.

### Step 2. Data preprocessing
<a class="anchor" id="chapter2"></a>

### Processing missing values

i will look at the columns in the table and look for missing values


In [2]:
family_status_unique = loanTable['family_status'].unique() #gives out unique family satus
family_status_valueCounts = loanTable['family_status'].value_counts() # counts family status values
family_status_valueCounts

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

the family status is devided in to 5 catagories. this is good becouse it can give me more resolution and can paint a bigger picture, but later i will also need to change the data to two main caragories: married, not married. so i can try and answer the basic question.

In [3]:
children_unique = loanTable['children'].unique() # gives out number of children values
family_status_valueCounts = loanTable['family_status'].value_counts() # counts family status values
loanTable.describe()
children_valueCounts = loanTable['children'].value_counts()
debt_valueCounts = loanTable['debt'].value_counts()

children_valueCounts

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

the number of children seems to have 47 cases of "-1", that can be a mistake in typing, there are 76 cases of 20 children that i can assume is a mistake as well maybe a typing mistake, all and all in this column there are 123 wierd values that I'll have to deal with. becouse it is a low % of the over all data and i cant be sure that this is a typo like -1 meaning 1 or 20 meaning 2.

In [4]:



children_median = loanTable['children'].median()
children_mean = loanTable['children'].mean()

'The median number of children is: {}, and the mean number of children is: {}'.format(children_median,children_mean) 

'The median number of children is: 0.0, and the mean number of children is: 0.5389082462253194'


i will replace these numbers with the median or the mean value, the mean value is not a whole number so i will use the median,it is a whole number, and they are close .

In [5]:
loanTable['children'].replace({-1:children_median,20:children_median},inplace=True) #replacing wierd values with the median of the column
children_valueCounts = loanTable['children'].value_counts()
children_valueCounts

0.0    14272
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

the "family_status" column seems to have no missing values.
The two columns i have nulls are "days_employed" and "total_income" and both have 2174 and they are the same rows that i put in a seperate dataframe called "loanTableNan".

In [15]:
loanTable.isnull().sum()
loanNan = loanTable.isnull()
loanNanRow = loanNan.any(axis=1)
loanTableNan = loanTable[loanNanRow]
loanTableNan.reset_index(inplace=True,drop=True)


loanTableNan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lowercase,purpose_catagory
0,0.0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,secondary education,wedding
1,0.0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education,secondary education,education
2,0.0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,secondary education,real astate
3,0.0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,secondary education,car
4,0.0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,secondary education,wedding


In [7]:
loanTable['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

The gender column has one 'XNA' value that is probably some sort of NaN that is not interfer with our task.
ill change the 'Xna' in the gender column choosing a random value with the same probability as we have in the column.


In [8]:



'''i will change the XNA in the gender column to either M or F at random with the same 
distribution chances as in the table'''

    
gender_value_sum = loanTable['gender'].value_counts().sum() # total number items in the gender column
male_number = loanTable[loanTable['gender']== 'M']['gender'].count() #calculates the number of 'M' in the column
female_number = loanTable[loanTable['gender']== 'F']['gender'].count() #calculates the number of 'F' in the column

randomNumber= random.randint(1,gender_value_sum) #genarates a random number between 1 and the total number items in the gender column
male_distribution = gender_value_sum - male_number
if randomNumber >= male_distribution:   
    loanTable.loc[loanTable['gender']=='XNA','gender']='M'  # assigns 'M' if randomNumber is in the distribution of the number of males
else:
     loanTable.loc[loanTable['gender']=='XNA','gender']='F' # assigns 'F' if randomNumber is in the distribution of the number of males
        

loanTable['gender'].value_counts()



F    14237
M     7288
Name: gender, dtype: int64



the 'total_income' column has missing values but i will not delete these columns just yet becouse i can stil use the other data in them.

The "debt" column has no missing values as well.



the 'education' column had lower and uppercase values i made a new column called 'education_lowercase' and changed all the values to lowercase.

# Data type replacement

In [9]:
# adding a new column with the 'education' in lowercase
loanTable['education_lowercase'] = loanTable['education'].str.lower()
loanTable

# adding a column with the catagories changes to only 4 values

loanTable['purpose_catagory'] = loanTable['purpose'].str.lower()


for row in range(len(loanTable)):
    if 'car' in loanTable.loc[row]['purpose_catagory']:
        loanTable.at[row,'purpose_catagory']= 'car'
    elif 'wedding' in loanTable.loc[row]['purpose_catagory']:
        loanTable.at[row,'purpose_catagory']= 'wedding'
    elif 'education' in loanTable.loc[row]['purpose_catagory'] or 'university' in loanTable.loc[row]['purpose_catagory']:
        loanTable.at[row,'purpose_catagory']= 'education'
    else:
        loanTable.at[row,'purpose_catagory']= 'real astate'

### Conclusion

the 'education' column had lower and uppercase values i made a new column called 'education_lowercase' and changed all the values to lowercase.

i can see that the easiest way to unify the 'purpose' column will have to be by words, there are 4 general purposes in this data frame: a wedding, buying real estate, buying a car or education. all the weddings have the word "wedding" in them all the cars have the word "car" in them the education has the word "education" or "university", and the rest is real estate. i can use this to classify into the 4 catagories. 


### Processing duplicates

### Conclusion

i did not find duplicates in this table

### Categorizing Data

### Conclusion

### Step 3. Answer these questions
<a class="anchor" id="chapter3"></a>

- Is there a relation between having kids and repaying a loan on time?

In [10]:
# children_pivot = loanTable.pivot_table(index=['debt'],columns = 'children',aggfunc= 'value_counts')
# children_pivot
debt_group = loanTable.groupby('debt')['children'].value_counts() # making a table that shows me by the number of kids if they have a debt or not

#gives me a value ratio between the number with x children whit no bedt to the x children with debt sorted 
number_of_children_debt_ratio = (debt_group[0]/debt_group[1]).sort_values() 
#number_of_children_debt_ratio

children_group = loanTable.groupby('children').agg({'debt':['count','sum']})

children_group['ratio']= children_group['debt','sum']/ children_group['debt','count']
children_group.sort_values(by= 'ratio', ascending = False) 

number_children_debt_table= loanTable[['children','debt']] # creating a new table just with 'children' and 'debt'
number_children_debt_table['have_kids']= '' # adding new column that will indicate if has kids or no

for row in range(len(number_children_debt_table)):
    if number_children_debt_table.loc[row]['children'] == 0:
        number_children_debt_table.at[row,'have_kids']= 'no'
    else:
        number_children_debt_table.at[row,'have_kids']= 'yes'

kids_yes_no_group = number_children_debt_table.groupby('have_kids').agg({'debt':['count','sum']})

kids_yes_no_group['ratio'] = kids_yes_no_group['debt','sum']/ kids_yes_no_group['debt','count']   

children_group.sort_values('ratio')
kids_yes_no_group    


debt           ratio
           count   sum          
have_kids                       
no         14272  1072  0.075112
yes         7253   669  0.092238

we can see that if we look just by the number of children (table 'children_group') there is no real relationship between the number of children to the debt, the 'inf' in the 5 children is becouse there is no one with 5 children that has a debt. so you might say that someone with 5 children will always pay their debt allthough ther is not enough of this value in the table to realy conclude this.
I calculated the relations between people with any number of children to no children at all, there is a slight advantage to not having children at all, around 7.5% with children have debt, and around 9.2% people with children have debt.



- Is there a relation between marital status and repaying a loan on time?

In [11]:
relation_status_grouped = loanTable.groupby('family_status').agg({'debt':['count','sum']})

relation_status_grouped['debt_ratio'] = relation_status_grouped['debt','sum']/ relation_status_grouped['debt','count']
relationship_status_table= loanTable[['family_status','family_status_id','debt']]
relationship_status_table['married']= ''

for row in range(len(relationship_status_table)):
    if relationship_status_table.loc[row]['family_status_id'] == 0:
        relationship_status_table.at[row,'married']= 'no'
    else:
        relationship_status_table.at[row,'married']= 'yes'
        
relationship_status_table_grouped = relationship_status_table.groupby('married').agg({'debt':['count','sum']})
relationship_status_table_grouped['ratio'] = relationship_status_table_grouped['debt','sum']/ relationship_status_table_grouped['debt','count']
relationship_status_table_grouped
relation_status_grouped.sort_values('debt_ratio')
relationship_status_table_grouped

debt          ratio
         count  sum          
married                      
no       12380  931  0.075202
yes       9145  810  0.088573

looking at the charts acourding to the relationship status (table: 'relation_status_grouped') it seems that the 'widow/widower' are the lowest in debt with around 6% other then that unmarried people are the worst with a 9.7%.

but when looking at the chart from a married of not point of veiw (table 'relationship_status_table_grouped')
not boing ,arried is preffrable with 7.5% in debt, and married with a 8.8% in debt.



- Is there a relation between income level and repaying a loan on time?

In [12]:
income_table = loanTable[['total_income','debt']]  
income_table =income_table.dropna().reset_index(drop=True)
mean_income = income_table['total_income'].mean()
mean_income_half = mean_income/2
mean_income_double = mean_income * 2

#adding a column that give the incom level 4 classes highest , med high, med low, low
income_table['income_rate']=''

for row in range(len(income_table)):
    if income_table.loc[row]['total_income'] >= mean_income_double:
        income_table.at[row,'income_rate']= 'highest'
    elif income_table.loc[row]['total_income'] < mean_income_double and income_table.loc[row]['total_income'] >= mean_income:
        income_table.at[row,'income_rate']= 'med high'
    elif income_table.loc[row]['total_income'] <= mean_income and income_table.loc[row]['total_income'] >= mean_income_half:
        income_table.at[row,'income_rate']= 'med low'
    else:
        income_table.at[row,'income_rate']= 'low'



In [13]:
income_table_grouped = income_table.groupby('income_rate').agg({'debt':['count','sum']})
income_table_grouped['ratio'] = income_table_grouped['debt','sum']/ income_table_grouped['debt','count']
income_table_grouped.sort_values('ratio')

debt          ratio
            count  sum          
income_rate                     
highest      1033   71  0.068732
low          2664  205  0.076952
med high     6426  497  0.077342
med low      9228  798  0.086476

### Conclusion

for this question i will adress the missing values in the income column, there are missing values in this column but i have no way of knowing what they are, replacing them with somthing will not help me becouse it will just give more eandom noise to the data so i made a new table ( 'income_table' ) and deleted the missing values. 
ill devide the incom to 4 classes using the mean  half of the mean and double the mean.
the table 'income_table_grouped' shows that people with verry high incom are more likely not to be in debt with a 6.8% in debt
but after that for the low and mid low range it is pretty close with 7.7% with debt. and with a 8.6% of the med high pay range are in debt. once again it is not a stright forward conclution

- How do different loan purposes affect on-time repayment of the loan?

In [14]:
purpose_table = loanTable[['purpose_catagory','debt']]
purpose_table_grouped = purpose_table.groupby('purpose_catagory').agg({'debt':['count','sum']})
purpose_table_grouped['ratio'] = purpose_table_grouped['debt','sum']/ purpose_table_grouped['debt','count']
purpose_table_grouped

debt          ratio
                  count  sum          
purpose_catagory                      
car                4315  403  0.093395
education          3610  331  0.091690
real astate       11252  821  0.072965
wedding            2348  186  0.079216

### Conclusion

when looking at the purpose of the loan 9.3% of people wanting a buy a car are in debt
not far behind them  educational purposes are at 9.1%
at ther bottom with 7.2% are people wanting to invest in real astate with 7.2% in debt

### Step 4. General conclusion
<a class="anchor" id="chapter4"></a>

from the data in this table i have reached the conclution that there is the optimal kind of person with the highest chance of paying back a loan. is a person that has no kids, not married or even better a widower, varry high paid (high pay for this purpose i calculated as double the mean and the total income) and the purpose of the loan is rel astate.
the precentage of change is not very high ranging a max of 3%. the lowest catagory for debt was a widower with 6.5% in debt.

the worst person to loan money to will be someone with kids ( 4 children is the worst) unmarried with a medium-high income (higher then the mean lower then double the mean) that wants to buy a car or education.
generaly people without kids will have a better chance of not having debt (7.5%) then with kids  (9.2%)
the one single value that is the worst is 9.7% with debt for people with 4 kids.

